In [1]:
!pip install opencv-python==4.4.0.40

In [1]:
import cv2
import numpy as np
import time 
import os
import matplotlib.pyplot as plt
import zipfile
print(cv2.__version__)
import winsound
import time

4.4.0


In [2]:
labels_path = os.path.sep.join(["obj.names"])
LABELS = open(labels_path).read().strip().split("\n")
weights_path = os.path.sep.join(["yolov4_custom_final.weights"])
config_path = os.path.sep.join(["yolov4_custom.cfg"])

net = cv2.dnn.readNet(config_path, weights_path)


In [3]:
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

ln = net.getLayerNames()
#print("Todas as camadas (layers):")
#print(ln)
#print("Total: "+ str(len(ln)))
#print("Camadas de saída: ")
#print(net.getUnconnectedOutLayers())
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
#print(ln)

In [4]:
def mostrar(img):
  fig = plt.gcf()
  fig.set_size_inches(16, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  plt.show()

In [5]:
def blob_imagem(net, imagem, mostrar_texto=True):
  inicio = time.time() 
  blob = cv2.dnn.blobFromImage(imagem, 1 / 255.0, (416, 416), swapRB=True, crop=False)
  net.setInput(blob)
  layerOutputs = net.forward(ln)
  termino = time.time()
  #if mostrar_texto:
    #print("YOLO levou {:.2f} segundos".format(termino - inicio))
  return net, imagem, layerOutputs

In [6]:
def deteccoes(detection, _threshold, caixas, confiancas, IDclasses):
  scores = detection[5:] 
  classeID = np.argmax(scores)  
  confianca = scores[classeID]

  if confianca > _threshold:
      caixa = detection[0:4] * np.array([W, H, W, H])     
      (centerX, centerY, width, height) = caixa.astype("int")
            
      x = int(centerX - (width / 2))
      y = int(centerY - (height / 2))

      caixas.append([x, y, int(width), int(height)])
      confiancas.append(float(confianca))
      IDclasses.append(classeID)
      
  return caixas, confiancas, IDclasses

In [7]:


def funcoes_imagem(imagem, i, confiancas, caixas, COLORS, LABELS, mostrar_texto=True):  
  (x, y) = (caixas[i][0], caixas[i][1])
  (w, h) = (caixas[i][2], caixas[i][3])

  cor = [int(c) for c in COLORS[IDclasses[i]]]
  cv2.rectangle(imagem, (x, y), (x + w, y + h), cor, 2) 
  texto = "{}: {:.4f}".format(LABELS[IDclasses[i]], confiancas[i])
  if mostrar_texto:
    print("> " + texto)
    print(x,y,w,h)
  cv2.putText(imagem, texto, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, cor, 2)

  return imagem,x,y,w,h

In [15]:
arquivo_video = 0
cap = cv2.VideoCapture(arquivo_video)
conectado, video = cap.read()

In [16]:
conectado

True

In [17]:
video_largura = video.shape[1]
video_altura = video.shape[0]
video_largura, video_altura

(640, 480)

In [18]:
def redimensionar(largura, altura, largura_maxima = 600): 
  if (largura > largura_maxima):
    proporcao = largura / altura
    video_largura = largura_maxima
    video_altura = int(video_largura / proporcao)
  else:
    video_largura = largura
    video_altura = altura

  return video_largura, video_altura

In [19]:
threshold = 0.5
threshold_NMS = 0.3
fonte_pequena, fonte_media = 0.4, 0.6
fonte = cv2.FONT_HERSHEY_SIMPLEX

In [20]:
classes = ["Helmet","Clothing","Glasses"]
while (cv2.waitKey(10) < 0):
  conectado, frame = cap.read()
  if not conectado:
    break
  t = time.time()
  frame = cv2.resize(frame, (video_largura, video_altura))
  try:
    (H, W) = frame.shape[:2]
  except:
    print('Erro')
    continue

  imagem_cp = frame.copy() 
  net, frame, layerOutputs = blob_imagem(net, frame)
  caixas = []       
  confiancas = []   
  IDclasses = []    

  for output in layerOutputs:
    for detection in output:
      caixas, confiancas, IDclasses = deteccoes(detection, threshold, caixas, confiancas, IDclasses)

  objs = cv2.dnn.NMSBoxes(caixas, confiancas, threshold, threshold_NMS)

  if len(objs) > 0:
    for i in objs.flatten():
          frame, x, y, w, h = funcoes_imagem(frame, i, confiancas, caixas, COLORS, LABELS, mostrar_texto=False)
          objeto = imagem_cp[y:y + h, x:x + w]
        
          if LABELS[IDclasses[i]] == "Helmet":  
            cv2.putText(frame, "COM CAPACETE", 
            (20, video_altura-400), fonte, 1, (200, 50, 250), 2, lineType=cv2.LINE_AA) 
                    
  
  cv2.putText(frame, "frame processado em {:.2f} segundos".format(time.time() - t), 
             (20, video_altura-20), fonte, fonte_pequena, (250, 250, 250), 0, lineType=cv2.LINE_AA)

  #if len(objs) == 0:
      #  cv2.putText(frame, "SEM CAPACETE", 
       # (20, video_altura-400), fonte, 1, (0, 50, 250), 2, lineType=cv2.LINE_AA)     
 
  cv2.imshow("tela",frame)

print('Terminou') 
 
cv2.destroyAllWindows()

Terminou
